In this notebook we first do the classification using the transformer This is our first classification task.

The output classification vector from the transformer is saved to be used by the FCNN This is our second classification task.


In [1]:
# Importing necessary libraries
import pandas as pd
from datetime import datetime
import sklearn
import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:
from simpletransformers.classification import ClassificationModel

In [3]:
input1 = torch.randn(1,4)
input2 = torch.randn(1,4)
output = torch.cosine_similarity(input1, input2)
print(input1)
print(input2)

print(output)

tensor([[-0.4101, -1.8006,  0.2481, -2.1183]])
tensor([[-0.3305,  0.6870,  1.8338, -0.9649]])
tensor([0.2243])


## Preparing the dataset

Some pre-processing to the dataset has already been done in preparation for various tests, so this processing is not from scratch.

In [4]:
# procedure for getting the data sets and formatting them for the transformer
 

def prepareDataset( filename):
     
    ReadSet=pd.read_excel(filename )

    ReadSet['text']=ReadSet['Statement']
    ReadSet['labels']=ReadSet['Label']
    
    ReadSet=ReadSet.drop(['ID','Label','Statement','Subject','Speaker','Job','From','Affiliation','PantsTotal','NotRealTotal','BarelyTotal','HalfTotal','MostlyTotal','Truths','Context'
],axis=1)
     
    return ReadSet


In [5]:
# preparing the training dataset
train=prepareDataset( 'trainRNDtext.xlsx')
# and display for inspecting
train

,text,labels
0,It costs more money to put a person on death r...,0
1,"Charlie Crist ""bizarrely vetoed"" $9.7 million ...",3
2,"In Massachusetts, half of the primary care doc...",4
3,Fulton County has successfully reduced the num...,4
4,Says he stood up to his own party by voting ag...,5
...,...,...
10264,Recently Rick Scott closed 30 womens health ca...,4
10265,Says Charlie Bass supports Paul Ryan plan that...,5
10266,A report by the US General Accountability Offi...,3
10267,"In the United States, weve had 12 years in a r...",1


In [6]:
# preparing the evaluation/validation dataset
Eval=prepareDataset('valid.xlsx')
# and display for inspecting
Eval

,text,labels
0,The president is brain-dead.,0
1,"Barack Obama supported keeping troops in Iraq,...",3
2,"He's leading by example, refusing contribution...",3
3,I'm the first person who really took up the is...,4
4,I built that border fence in San Diego...and i...,4
...,...,...
1279,CNN accidentally aired 30 minutes of pornograp...,1
1280,President Obamas American Recovery and Reinves...,2
1281,We (in Illinois) have the fifth-highest tax bu...,4
1282,Says Donald Trump won more counties than any c...,4


In [7]:
# preparing the test set dataset
test=prepareDataset('test.xlsx')
test

,text,labels
0,New Mexico was 46th in teacher pay (when he wa...,4
1,Barack Obama and Hillary Clinton have changed ...,3
2,I'll tell you what I can tell this country: If...,1
3,Tommy Thompson created the first school choice...,5
4,Fifty-six percent decline in overall crime. A ...,5
...,...,...
1278,"We have trade agreements with 20 countries, an...",1
1279,On Donald Trumps plan to cut federal funding t...,4
1280,"Black Lives Matter, who are attacking law enfo...",2
1281,Latina who enthusiastically supported Donald T...,0


## Setting up the transformer for fine tuning

This is where changes are done to optimise the model

The simpletransformers library is the quickest way to do this at the time of writing. 
For more information on the settings and their default value go here:
https://github.com/ThilinaRajapakse/simpletransformers#default-settings 

###### Please do read that reference before changing any parameters. Don't try to be a hero!

In [8]:
#Set the model being used here
model_class='albert'  # bert or roberta or albert
model_version='albert-large-v2' #bert-base-cased, roberta-base, roberta-large, albert-base-v2 OR albert-large-v2


output_folder='./TunedModels/'+model_class+'/'+model_version+"/"
cache_directory= "./TunedModels/"+model_class+"/"+model_version+"/cache/"
labels_count=6  # the number of classification classes

print('model variables were set up: ')

model variables were set up: 


In [9]:
# use this to test if writing to the directories is working

import os
print(os.getcwd())
print(output_folder)
print(cache_directory)

testWrite=train.head(30)
 
testWrite.to_csv(output_folder+'DeleteThisToo.tsv', sep='\t')
testWrite.to_csv(cache_directory+'DeleteThisToo.tsv', sep='\t')

del(testWrite)

G:\0 finalThesis\randomText
./TunedModels/albert/albert-large-v2/
./TunedModels/albert/albert-large-v2/cache/


In [15]:
 
save_every_steps=1285
# assuming training batch size of 8
# any number above 1284 saves the model only at every epoch
# Saving the model mid training very often will consume disk space fast

train_args={
    "output_dir":output_folder,
    "cache_dir":cache_directory,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 1,
    "save_steps": save_every_steps, 
    "learning_rate": 2e-5,
    "train_batch_size": 64,
    "eval_batch_size": 8,
    "evaluate_during_training_steps": 312,
    "max_seq_length": 64,
    "n_gpu": 1,
}

# Create a ClassificationModel
model = ClassificationModel(model_class, model_version, num_labels=labels_count, args=train_args) 

# You can set class weights by using the optional weight argument

### Loading a saved model (based on above args{})

If you stopped training you can continue training from a previously saved check point.
The next cell allows you to load a model from any checkpoint.
The number of epochs in the train_args{} will be done and continue tuning from your checkpoint.

###### HOWEVER
It will overwrite previous checkpoints!
Example:  If you load an epoch-3 checkpoint, the epoch-1 checkpoint will be overwritten by the 4th epoch and it will be equivalent to a 4th epoch even if you have epoch-1 in the name.
###### SO BE CAREFUL

In [22]:
# loading a previously saved model based on this particular Transformer Class and model_name

# loading the checkpoint that gave the best result
CheckPoint='checkpoint-322-epoch-2'  #epoch 2


preSavedCheckpoint=output_folder+CheckPoint

print('Loading model, please wait...')
model = ClassificationModel( model_class, preSavedCheckpoint, num_labels=labels_count, args=train_args) 
print('model in use is :', preSavedCheckpoint )

Loading model, please wait...
model in use is : ./TunedModels/albert/albert-large-v2/checkpoint-322-epoch-2


## Training the Transformer

Skip the next cell if you want to skip the training and go directly to the evaluation

In [18]:
# Train the model
current_time = datetime.now()
model.train_model(train)
print("Training time: ", datetime.now() - current_time)

Converting to features started. Cache is not used.



Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 1.689072

Training of albert model complete. Saved to ./TunedModels/albert/albert-large-v2/.
Training time:  0:03:00.334546


## Evaluating the training

In [23]:
TrainResult, TrainModel_outputs, wrong_predictions = model.eval_model(train, acc=sklearn.metrics.accuracy_score)

EvalResult, EvalModel_outputs, wrong_predictions = model.eval_model(Eval, acc=sklearn.metrics.accuracy_score)

TestResult, TestModel_outputs, wrong_predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score)

print('Training Result:', TrainResult['acc'])
#print('Model Out:', TrainModel_outputs)

print('Eval Result:', EvalResult['acc'])
#print('Model Out:', EvalModel_outputs)

print('Test Set Result:', TestResult['acc'])
#print('Model Out:', TestModel_outputs)

Features loaded from cache at ./TunedModels/albert/albert-large-v2/cache/cached_dev_albert_64_6_10269



{'mcc': 0.03041130547938583, 'acc': 0.2162820138280261, 'eval_loss': 1.7519530008700779}
Features loaded from cache at ./TunedModels/albert/albert-large-v2/cache/cached_dev_albert_64_6_1284



{'mcc': -0.009363452126874507, 'acc': 0.19080996884735202, 'eval_loss': 1.7674117969430012}
Features loaded from cache at ./TunedModels/albert/albert-large-v2/cache/cached_dev_albert_64_6_1283



{'mcc': -0.010929883051276498, 'acc': 0.2034294621979735, 'eval_loss': 1.7521654197147913}
Training Result: 0.2162820138280261
Eval Result: 0.19080996884735202
Test Set Result: 0.2034294621979735


In [24]:
Pred=[]

countCorrect=0

for row in range(TestModel_outputs.shape[0]):
    outputs=TestModel_outputs[row]
    #print(test.iloc[row,0])
    print(outputs, end=' ')
    
    result=0
    if outputs[0]<outputs[1]:result=1
    if outputs[result]<outputs[2]:result=2
    if outputs[result]<outputs[3]:result=3
    if outputs[result]<outputs[4]:result=4
    if outputs[result]<outputs[5]:result=5
    Pred.append(result)
    print(result, ' ',test.iloc[row,1], end=' ')
    if result==test.iloc[row,1]:
        countCorrect+=1
        print('Match',countCorrect)
    print('')

print(countCorrect)
#Pred

[-0.73779297  0.20373535 -0.0227356   0.31225586  0.0958252   0.05310059] 3   4 
[-0.68847656  0.2512207   0.01332092  0.2836914   0.10888672  0.02445984] 3   3 Match 1

[-0.7558594   0.2421875  -0.01084137  0.3046875   0.0793457   0.01844788] 3   1 
[-0.71484375  0.23608398  0.00139999  0.32177734  0.10412598  0.03207397] 3   5 
[-0.59521484  0.3383789   0.0607605   0.32617188  0.15124512 -0.00679779] 1   5 
[-0.63378906  0.22302246  0.01179504  0.32763672  0.14550781  0.05947876] 3   2 
[-0.7651367   0.19580078 -0.0244751   0.34277344  0.08435059  0.04986572] 3   4 
[-0.7451172   0.30639648 -0.07012939  0.31298828  0.08636475 -0.04708862] 3   5 
[-0.9111328   0.2467041  -0.19567871  0.33276367  0.16955566  0.02609253] 3   4 
[-0.78027344  0.21154785 -0.0355835   0.3708496   0.03652954  0.07940674] 3   5 
[-0.8076172   0.29003906 -0.11419678  0.37451172  0.02505493 -0.0461731 ] 3   5 
[-0.66748047  0.3046875   0.02294922  0.2763672   0.16967773 -0.00335121] 1   3 
[-0.72509766  0.2244

[-0.68066406  0.26708984  0.01407623  0.28857422  0.1361084  -0.01681519] 3   0 
[-0.7421875   0.33520508 -0.08044434  0.3552246   0.11096191 -0.04754639] 3   2 
[-0.734375    0.2043457  -0.02694702  0.30664062  0.07592773  0.03540039] 3   5 
[-0.7470703   0.18383789 -0.00684357  0.34204102  0.09222412  0.07897949] 3   4 
[-0.74658203  0.22314453 -0.05282593  0.30688477  0.05981445  0.05065918] 3   1 
[-0.6699219   0.2442627  -0.03092957  0.3203125   0.1352539  -0.05130005] 3   0 
[-0.7363281   0.20471191 -0.0218811   0.3059082   0.11791992  0.07196045] 3   5 
[-0.76708984  0.29907227 -0.0758667   0.33935547  0.09185791 -0.0447998 ] 3   1 
[-0.73046875  0.2536621  -0.03295898  0.31689453  0.11108398  0.00453186] 3   1 
[-0.7265625   0.21984863 -0.05303955  0.34521484  0.05908203  0.04354858] 3   3 Match 20

[-0.7661133   0.19641113 -0.0329895   0.30566406  0.07092285  0.06951904] 3   2 
[-0.7504883   0.19458008 -0.06561279  0.31347656  0.11541748  0.05044556] 3   2 
[-0.70996094  0.262

[-0.7416992   0.22265625 -0.03390503  0.29296875  0.09509277  0.03497314] 3   1 
[-0.6040039   0.2800293   0.05691528  0.30371094  0.19091797 -0.00535965] 3   3 Match 49

[-0.70458984  0.18395996 -0.05044556  0.32739258  0.06445312  0.05926514] 3   5 
[-0.7290039   0.18933105 -0.03117371  0.32666016  0.07318115  0.04653931] 3   4 
[-0.67871094  0.2475586  -0.00233078  0.3190918   0.1036377   0.03839111] 3   1 
[-0.7265625   0.21765137 -0.05679321  0.32543945  0.0970459   0.0737915 ] 3   0 
[-0.8027344   0.14367676 -0.0869751   0.32348633  0.03903198  0.04925537] 3   1 
[-0.66503906  0.2590332   0.02285767  0.28735352  0.12414551  0.05374146] 3   3 Match 50

[-0.67285156  0.23291016  0.00088406  0.28735352  0.11883545  0.05755615] 3   2 
[-0.70214844  0.22473145 -0.05307007  0.30371094  0.09667969  0.01390076] 3   2 
[-0.7026367   0.23303223 -0.03442383  0.2939453   0.14892578  0.0296936 ] 3   5 
[-0.69140625  0.20410156 -0.05715942  0.3305664   0.08728027  0.0496521 ] 3   3 Match 51

[

[-0.80615234  0.23620605 -0.0894165   0.34936523  0.05197144  0.00721359] 3   0 
[-0.765625    0.24487305 -0.08178711  0.33081055  0.0557251   0.0098877 ] 3   1 
[-0.73046875  0.21240234 -0.04821777  0.33081055  0.07977295  0.09130859] 3   5 
[-0.74609375  0.22241211 -0.0435791   0.3178711   0.11053467  0.05831909] 3   4 
[-0.69921875  0.19372559 -0.02331543  0.3310547   0.09246826  0.07867432] 3   5 
[-0.7241211   0.23657227 -0.04452515  0.33276367  0.06536865  0.06341553] 3   1 
[-0.7128906   0.34399414 -0.08209229  0.31640625  0.28759766 -0.09570312] 1   3 
[-0.75439453  0.19299316 -0.05715942  0.35913086  0.08892822  0.05474854] 3   2 
[-0.75683594  0.2175293  -0.01965332  0.33032227  0.06793213  0.03668213] 3   2 
[-0.7158203   0.24853516 -0.00390434  0.28198242  0.11743164  0.02684021] 3   1 
[-0.7104492   0.20581055  0.02284241  0.31518555  0.1385498   0.04937744] 3   4 
[-0.7216797   0.18249512 -0.05578613  0.35595703  0.07012939  0.08666992] 3   1 
[-0.7504883   0.20874023 -0.

[-0.68603516  0.21984863 -0.01812744  0.31591797  0.11724854  0.05090332] 3   1 
[-0.7050781   0.24206543 -0.07946777  0.31982422  0.09405518 -0.04904175] 3   3 Match 112

[-0.7036133   0.25952148  0.00407791  0.32861328  0.07220459  0.0322876 ] 3   5 
[-0.6816406   0.27124023 -0.01472473  0.27294922  0.10009766  0.01654053] 3   3 Match 113

[-0.6220703   0.2298584  -0.02510071  0.28027344  0.13000488  0.07995605] 3   1 
[-0.74902344  0.23156738 -0.0758667   0.3581543   0.06762695  0.05776978] 3   4 
[-0.59472656  0.33154297  0.01397705  0.29223633  0.16821289  0.05038452] 1   3 
[-0.74560547  0.24584961 -0.01348877  0.32641602  0.09698486  0.03890991] 3   3 Match 114

[-0.68847656  0.2277832  -0.00590515  0.32666016  0.08514404  0.04934692] 3   3 Match 115

[-0.7495117   0.25317383 -0.08862305  0.34521484  0.08050537  0.0032959 ] 3   5 
[-0.7866211   0.17248535 -0.07299805  0.35839844  0.04769897  0.05377197] 3   1 
[-0.6953125   0.19042969 -0.04199219  0.2939453   0.08648682  0.05749

...

[-0.77197266  0.22485352 -0.03982544  0.34423828  0.11114502  0.06262207] 3   3 Match 165

[-0.71728516  0.19311523 -0.03353882  0.3232422   0.08319092  0.03884888] 3   2 
[-0.73291016  0.21850586 -0.04812622  0.31640625  0.1204834   0.01899719] 3   5 
[-0.7788086   0.18945312 -0.06082153  0.32739258  0.0880127   0.02964783] 3   5 
[-0.72753906  0.24829102 -0.07855225  0.32177734  0.10101318 -0.00466156] 3   1 
[-0.76416016  0.25219727 -0.06756592  0.30639648  0.08410645  0.00565338] 3   1 
[-0.75146484  0.27319336 -0.05010986  0.30395508  0.10137939  0.04742432] 3   3 Match 166

[-0.68310547  0.23510742 -0.01167297  0.35375977  0.07397461  0.05114746] 3   2 
[-0.7089844   0.20605469 -0.05126953  0.32373047  0.09576416  0.07867432] 3   5 
[-0.6723633   0.23815918  0.01197815  0.33007812  0.08953857  0.04541016] 3   2 
[-0.7451172   0.22570801 -0.00251198  0.31567383  0.1373291   0.0567627 ] 3   1 
[-0.7651367   0.17956543 -0.11010742  0.3852539   0.07061768  0.09356689] 3   3 Match 167


[-0.7241211   0.18066406 -0.04470825  0.3125      0.06512451  0.06115723] 3   0 
[-0.73583984  0.2211914  -0.00443649  0.32128906  0.09625244  0.01519012] 3   4 
[-0.69970703  0.28149414 -0.02464294  0.32910156  0.07366943  0.04260254] 3   4 
[-0.73779297  0.2084961  -0.07055664  0.3215332   0.0836792   0.04406738] 3   5 
[-0.75878906  0.18688965 -0.0191803   0.3166504   0.0748291   0.03549194] 3   3 Match 188

[-0.6767578   0.27612305 -0.01283264  0.29174805  0.13012695  0.0304718 ] 3   2 
[-0.7519531   0.2097168  -0.0164032   0.30639648  0.10040283 -0.01178741] 3   0 
[-0.66064453  0.26831055 -0.01013947  0.28051758  0.10498047  0.03421021] 3   4 
[-0.7866211   0.26635742 -0.08520508  0.32495117  0.07415771  0.06451416] 3   2 
[-0.6503906   0.25732422 -0.02902222  0.31567383  0.16137695  0.02125549] 3   2 
[-0.73779297  0.24035645 -0.07122803  0.3256836   0.06542969  0.0345459 ] 3   2 
[-0.7265625   0.25805664 -0.07470703  0.3310547   0.09545898  0.05111694] 3   4 
[-6.8554688e-01  

[-0.6743164   0.24560547 -0.0380249   0.33911133  0.10021973  0.03970337] 3   0 
[-0.75341797  0.22924805 -0.03111267  0.3239746   0.07208252  0.06634521] 3   1 
[-0.73095703  0.21826172 -0.05102539  0.35498047  0.09069824  0.07434082] 3   5 
[-0.64404297  0.3244629   0.01525879  0.25073242  0.1685791   0.0383606 ] 1   2 
[-0.7753906   0.19018555 -0.04928589  0.39111328  0.07336426  0.09204102] 3   3 Match 214

[-0.56689453  0.3540039   0.04141235  0.28710938  0.19421387 -0.00075054] 1   4 
[-0.7373047   0.18457031 -0.05386353  0.3317871   0.04321289  0.05407715] 3   2 
[-0.77197266  0.18896484 -0.10827637  0.33422852  0.02577209  0.00584412] 3   1 
[-0.7290039   0.23352051 -0.04464722  0.3347168   0.08129883  0.0340271 ] 3   3 Match 215

[-0.6660156   0.27954102  0.0013237   0.26831055  0.14672852  0.02420044] 1   4 
[-0.77001953  0.1862793  -0.01501465  0.31689453  0.09265137  0.06192017] 3   1 
[-0.68310547  0.2607422  -0.01012421  0.29833984  0.08587646 -0.0076828 ] 3   3 Match 216

[-0.6044922   0.2956543   0.05236816  0.28125     0.18688965  0.01425171] 1   4 
[-0.7246094   0.22143555 -0.02952576  0.32592773  0.07403564  0.06469727] 3   4 
[-0.72509766  0.15710449 -0.11071777  0.36132812  0.06884766  0.13452148] 3   2 
[-0.75        0.16149902 -0.05700684  0.33496094  0.06982422  0.05307007] 3   2 
[-0.7114258   0.21447754 -0.00648117  0.32641602  0.1038208   0.0380249 ] 3   2 
[-0.71240234  0.19689941 -0.02619934  0.33984375  0.11260986  0.07275391] 3   2 
[-0.7792969   0.2076416  -0.08239746  0.29492188  0.02375793 -0.01380157] 3   3 Match 237

[-0.7324219   0.2541504  -0.01544952  0.3149414   0.08380127  0.01794434] 3   4 
[-0.75        0.21826172 -0.05493164  0.34350586  0.04837036  0.06268311] 3   0 
[-0.6791992   0.21740723 -0.04846191  0.30200195  0.08581543  0.06945801] 3   3 Match 238

[-0.7866211   0.2619629  -0.10186768  0.35498047  0.10565186 -0.02290344] 3   5 
[-0.67089844  0.31420898  0.01834106  0.35083008  0.13354492  0.00780869] 3   3 Match 239

[-0.66552734  0.22253418 -0.03692627  0.29785156  0.09625244  0.05340576] 3   4 
[-0.7114258   0.28710938 -0.04708862  0.3413086   0.09912109  0.00397491] 3   4 
[-0.68652344  0.19470215  0.00685501  0.31054688  0.13269043  0.05456543] 3   2 
[-0.71875     0.18017578 -0.01980591  0.36108398  0.08129883  0.05566406] 3   3 Match 260

[-0.8017578   0.22631836 -0.03050232  0.3371582   0.07177734  0.03579712] 3   2 
[-0.68847656  0.20153809 -0.03921509  0.29492188  0.10144043  0.08050537] 3   5 
[-0.7548828   0.19543457 -0.06228638  0.3251953   0.08026123  0.05081177] 3   1 
[-0.69921875  0.19042969 -0.0134964   0.30908203  0.09936523  0.05618286] 3   2 
[-0.6660156   0.22619629 -0.02914429  0.34985352  0.08618164  0.08868408] 3   0 
[-0.7192383   0.23620605 -0.01470947  0.3071289   0.06707764  0.04785156] 3   0 
[-0.7089844   0.20629883 -0.08679199  0.32666016  0.0647583   0.06591797] 3   3 Match 261

[-0.7680664   0.17553711 -0.03842163  0.3564453   0.08831787  0.01091003] 3   2 
[-0.7553

In [25]:
from sklearn import metrics
 
print(metrics.confusion_matrix(test['labels'],Pred))

[[  0   5   0  87   0   0]
 [  0  13   0 237   0   0]
 [  0  18   0 196   0   0]
 [  0  19   0 248   0   0]
 [  0  26   0 223   0   0]
 [  0  15   0 196   0   0]]


In [26]:
target_names = ['Pants', 'False', 'Barely-True','Half-True','Mostly-True','True']

print(metrics.classification_report(test['labels'], Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.00      0.00      0.00        92
       False       0.14      0.05      0.08       250
 Barely-True       0.00      0.00      0.00       214
   Half-True       0.21      0.93      0.34       267
 Mostly-True       0.00      0.00      0.00       249
        True       0.00      0.00      0.00       211

    accuracy                           0.20      1283
   macro avg       0.06      0.16      0.07      1283
weighted avg       0.07      0.20      0.09      1283



C:\Users\Mark\Anaconda3\envs\thesis\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
# saving the output of the models to CSVs
#these are 1X6 classification vectors

SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/Saves/"
print('Saving...')
trainOut = pd.DataFrame(data= TrainModel_outputs )
trainOut.to_csv(SavesDirectory+'trainOut.tsv', sep='\t',  index=False)

evalOut = pd.DataFrame(data= EvalModel_outputs )
evalOut.to_csv(SavesDirectory+'evalOut.tsv', sep='\t',  index=False)

testOut = pd.DataFrame(data= TestModel_outputs )
testOut.to_csv(SavesDirectory+'testOut.tsv', sep='\t',  index=False)

print('Saving Complete on',datetime.now() ,'in:', SavesDirectory)

Saving...
Saving Complete on 2020-03-30 18:53:43.668869 in: ./TunedModels/albert/albert-large-v2/Saves/


In [28]:
del(model)
#del(train,Eval,test)
del(trainOut,evalOut,testOut)
torch.cuda.empty_cache()

#  Adding the reputation vector

This section takes the output results from the transformer used above and uses it together with the speaker's reputation to enhance the classification.

Before running this section it is suggested that you halt the program and start running it again from this cell. The neural net will likely have an error caused by some unreleased variable used by thr simple transformers library. 

In [1]:
import pandas as pd
import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:

train=pd.read_excel('trainReputation.xlsx' )
train=train.iloc[:,:-2].astype(float)
train=train/200  #for scaling
#train

model_class='albert'  # bert or roberta or albert
model_version='albert-large-v2' #bert-base-cased, roberta-base, roberta-large, albert-base-v2 OR albert-large-v2
SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/Saves/"
TF_Output=pd.read_csv( SavesDirectory+'trainOut.tsv', sep='\t')

train=pd.concat([train,TF_Output], axis=1)

train

,PantsTotal,NotRealTotal,BarelyTotal,HalfTotal,MostlyTotal,Truths,0,1,2,3,4,5
0,0.005,0.000,0.000,0.000,0.000,0.000,-0.723145,0.213379,0.000787,0.322021,0.105225,0.046814
1,0.095,0.160,0.170,0.290,0.165,0.165,-0.657227,0.261475,-0.007530,0.341064,0.132202,-0.009216
2,0.005,0.010,0.005,0.015,0.040,0.010,-0.711426,0.196411,-0.025452,0.342285,0.091919,0.047363
3,0.005,0.010,0.005,0.015,0.040,0.010,-0.744629,0.279541,-0.048859,0.376221,0.030746,-0.011841
4,0.035,0.145,0.200,0.345,0.380,0.365,-0.687012,0.213623,-0.060669,0.288818,0.093445,0.036285
...,...,...,...,...,...,...,...,...,...,...,...,...
10264,0.005,0.030,0.070,0.050,0.050,0.020,-0.701172,0.222290,-0.032501,0.326172,0.062439,0.044952
10265,0.055,0.075,0.080,0.100,0.050,0.035,-0.699707,0.165649,-0.015297,0.312988,0.124878,0.049164
10266,0.035,0.115,0.140,0.190,0.170,0.075,-0.642578,0.189941,-0.032532,0.315674,0.144043,0.040405
10267,0.305,0.570,0.315,0.255,0.185,0.070,-0.728516,0.233398,0.000734,0.306885,0.088257,-0.003422


In [3]:
TrainLables=pd.read_excel('trainReputation.xlsx' )
TrainLables=TrainLables.iloc[:,-1] 

TrainLables=pd.get_dummies(TrainLables)
TrainLables

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,0,0,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,0,1
...,...,...,...,...,...,...
10264,0,0,0,0,1,0
10265,0,0,0,0,0,1
10266,0,0,0,1,0,0
10267,0,1,0,0,0,0


In [4]:
input=torch.tensor(train.values)
del(train)
input

tensor([[ 0.0050,  0.0000,  0.0000,  ...,  0.3220,  0.1052,  0.0468],
        [ 0.0950,  0.1600,  0.1700,  ...,  0.3411,  0.1322, -0.0092],
        [ 0.0050,  0.0100,  0.0050,  ...,  0.3423,  0.0919,  0.0474],
        ...,
        [ 0.0350,  0.1150,  0.1400,  ...,  0.3157,  0.1440,  0.0404],
        [ 0.3050,  0.5700,  0.3150,  ...,  0.3069,  0.0883, -0.0034],
        [ 0.0000,  0.0050,  0.0000,  ...,  0.3440,  0.0863,  0.0205]],
       dtype=torch.float64)

In [5]:
targets=torch.tensor(TrainLables.astype(float).values)
del(TrainLables)
targets

tensor([[1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        ...,
        [0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.]], dtype=torch.float64)

In [6]:
 
size= torch.tensor(input[0].size())
InputSize=size.item()

OutputSize=torch.tensor(targets[0].size()).item()

print('input size:', InputSize)
print('output size:', OutputSize)

input size: 12
output size: 6


In [7]:

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
         
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(InputSize, 24)  # input size 
        self.fc2 = nn.Linear(24,12)
        self.fc3 = nn.Linear(12, OutputSize)  #classifies 'outputsize' different classes

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x)) 
        x = torch.tanh(self.fc3(x)).double()
        return x

    

#now we use it

net = Net()

In [8]:
# here we  setup the neural network parameters
# pick an optimizer (Simple Gradient Descent)

learning_rate = 10e-4
criterion = nn.MSELoss()  #computes the loss Function

import torch.optim as optim

# creating optimizer
#optimizer = optim.SGD(net.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)


In [9]:
for epoch in range(3000):  
        
    optimizer.zero_grad()   # zero the gradient buffers
    output = net(input.float())

    loss = criterion(output, targets)
    print('Loss:', loss, ' at epoch:', epoch)

    loss.backward()  #backprop
    optimizer.step()    # Does the update

Loss: tensor(0.1515, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 0
Loss: tensor(0.1502, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 1
Loss: tensor(0.1490, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2
Loss: tensor(0.1479, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 3
Loss: tensor(0.1468, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 4
Loss: tensor(0.1459, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 5
Loss: tensor(0.1451, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 6
Loss: tensor(0.1443, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 7
Loss: tensor(0.1436, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 8
Loss: tensor(0.1429, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 9
Loss: tensor(0.1423, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 10
Loss: tensor(0.1417, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 11
Loss: tensor(0

Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 100
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 101
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 102
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 103
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 104
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 105
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 106
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 107
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 108
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 109
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 110
Loss: tensor(0.1357, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.1351, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 206
Loss: tensor(0.1351, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 207
Loss: tensor(0.1351, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 208
Loss: tensor(0.1351, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 209
Loss: tensor(0.1351, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 210
Loss: tensor(0.1351, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 211
Loss: tensor(0.1351, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 212
Loss: tensor(0.1351, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 213
Loss: tensor(0.1351, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 214
Loss: tensor(0.1351, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 215
Loss: tensor(0.1351, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 216
Loss: tensor(0.1350, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.1345, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 311
Loss: tensor(0.1345, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 312
Loss: tensor(0.1345, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 313
Loss: tensor(0.1345, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 314
Loss: tensor(0.1345, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 315
Loss: tensor(0.1345, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 316
Loss: tensor(0.1345, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 317
Loss: tensor(0.1345, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 318
Loss: tensor(0.1344, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 319
Loss: tensor(0.1344, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 320
Loss: tensor(0.1344, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 321
Loss: tensor(0.1344, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

Loss: tensor(0.1338, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 416
Loss: tensor(0.1338, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 417
Loss: tensor(0.1338, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 418
Loss: tensor(0.1338, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 419
Loss: tensor(0.1338, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 420
Loss: tensor(0.1338, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 421
Loss: tensor(0.1338, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 422
Loss: tensor(0.1338, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 423
Loss: tensor(0.1338, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 424
Loss: tensor(0.1338, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 425
Loss: tensor(0.1338, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 426
Loss: tensor(0.1338, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epo

...

Loss: tensor(0.1085, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2430
Loss: tensor(0.1085, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2431
Loss: tensor(0.1084, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2432
Loss: tensor(0.1084, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2433
Loss: tensor(0.1084, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2434
Loss: tensor(0.1084, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2435
Loss: tensor(0.1084, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2436
Loss: tensor(0.1084, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2437
Loss: tensor(0.1084, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2438
Loss: tensor(0.1084, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2439
Loss: tensor(0.1084, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2440
Loss: tensor(0.1084, dtype=torch.float64, grad_fn=<MseLossBackwar

Loss: tensor(0.1075, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2550
Loss: tensor(0.1075, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2551
Loss: tensor(0.1075, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2552
Loss: tensor(0.1075, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2553
Loss: tensor(0.1075, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2554
Loss: tensor(0.1075, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2555
Loss: tensor(0.1075, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2556
Loss: tensor(0.1074, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2557
Loss: tensor(0.1074, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2558
Loss: tensor(0.1074, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2559
Loss: tensor(0.1074, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2560
Loss: tensor(0.1074, dtype=torch.float64, grad_fn=<MseLossBackwar

Loss: tensor(0.1067, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2670
Loss: tensor(0.1067, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2671
Loss: tensor(0.1067, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2672
Loss: tensor(0.1067, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2673
Loss: tensor(0.1067, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2674
Loss: tensor(0.1067, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2675
Loss: tensor(0.1066, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2676
Loss: tensor(0.1066, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2677
Loss: tensor(0.1066, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2678
Loss: tensor(0.1066, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2679
Loss: tensor(0.1066, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2680
Loss: tensor(0.1066, dtype=torch.float64, grad_fn=<MseLossBackwar

Loss: tensor(0.1060, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2792
Loss: tensor(0.1060, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2793
Loss: tensor(0.1060, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2794
Loss: tensor(0.1060, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2795
Loss: tensor(0.1060, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2796
Loss: tensor(0.1060, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2797
Loss: tensor(0.1060, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2798
Loss: tensor(0.1060, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2799
Loss: tensor(0.1060, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2800
Loss: tensor(0.1060, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2801
Loss: tensor(0.1059, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2802
Loss: tensor(0.1059, dtype=torch.float64, grad_fn=<MseLossBackwar

Loss: tensor(0.1054, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2913
Loss: tensor(0.1054, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2914
Loss: tensor(0.1054, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2915
Loss: tensor(0.1054, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2916
Loss: tensor(0.1054, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2917
Loss: tensor(0.1054, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2918
Loss: tensor(0.1054, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2919
Loss: tensor(0.1054, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2920
Loss: tensor(0.1054, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2921
Loss: tensor(0.1054, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2922
Loss: tensor(0.1054, dtype=torch.float64, grad_fn=<MseLossBackward>)  at epoch: 2923
Loss: tensor(0.1054, dtype=torch.float64, grad_fn=<MseLossBackwar

In [26]:
#save the FCNN model

stage='NNetwork6Way/'
SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/"+stage
PATH = SavesDirectory+' name of saved model here .pth'

torch.save(net.state_dict(), PATH)

# more on saving pytorch networks: https://pytorch.org/docs/stable/notes/serialization.html

In [10]:
#load previously saved FCNN model 

stage='NNetwork6Way/'
SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/"+stage
#PATH = SavesDirectory+'Tanh_MSE_adam4271.pth'

net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [10]:
# load the test data

TestData=pd.read_excel('testReputation.xlsx' )
TestData=TestData.iloc[:,:-2].astype(float)
TestData=TestData/200

SavesDirectory='./TunedModels/'+model_class+'/'+model_version+"/Saves/"
TF_Output=pd.read_csv( SavesDirectory+'testOut.tsv', sep='\t')

In [11]:


TestData=pd.concat([TestData,TF_Output], axis=1)
TestData=torch.tensor(TestData.values)
TestData
 

tensor([[ 0.0050,  0.0100,  0.0050,  ...,  0.3123,  0.0958,  0.0531],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.2837,  0.1089,  0.0245],
        [ 0.0200,  0.0250,  0.0600,  ...,  0.3047,  0.0793,  0.0184],
        ...,
        [ 0.0100,  0.0050,  0.0250,  ...,  0.3257,  0.1226,  0.0760],
        [ 0.2200,  0.0950,  0.0350,  ...,  0.3313,  0.1141,  0.0054],
        [ 0.0050,  0.0600,  0.0100,  ...,  0.3450,  0.1208, -0.0110]],
       dtype=torch.float64)

In [12]:
labels=pd.read_excel('testReputation.xlsx' )
labels=labels.iloc[:,-1] 
labelsOneHot=pd.get_dummies(labels)
labelsOneHot

,0,1,2,3,4,5
0,0,0,0,0,1,0
1,0,0,0,1,0,0
2,0,1,0,0,0,0
3,0,0,0,0,0,1
4,0,0,0,0,0,1
...,...,...,...,...,...,...
1278,0,1,0,0,0,0
1279,0,0,0,0,1,0
1280,0,0,1,0,0,0
1281,1,0,0,0,0,0


In [13]:
TestLables =torch.tensor(labelsOneHot.values)
TestLables

tensor([[0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0],
        ...,
        [0, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0]], dtype=torch.uint8)

In [14]:
 correct = 0
total = 0


Y=[]  #target
Pred=[]  #predicted

with torch.no_grad():
    for row in range(len(TestData)):
        outputs = net(TestData[row,:].float())
        result=0
        total+=1
        if outputs[0]<outputs[1]:result=1
        if outputs[result]<outputs[2]:result=2
        if outputs[result]<outputs[3]:result=3
        if outputs[result]<outputs[4]:result=4
        if outputs[result]<outputs[5]:result=5
        
        if labelsOneHot.iloc[row,result]==1: correct+=1
        
        Y.append(result)
        Pred.append(labels.iloc[row])
        
        print(result, end=' ')
        
       
print('Correct:', correct, 'out of:', total )
print('Accuracy of the network : ',( 100 * correct / total))

4 4 3 1 3 3 4 4 4 4 4 3 3 5 3 3 1 3 3 4 4 4 3 3 3 4 3 4 3 4 4 3 0 4 4 3 4 4 4 4 1 3 4 1 4 4 3 4 4 4 4 3 3 3 3 4 5 2 4 3 3 3 4 4 1 4 3 3 5 4 3 1 3 3 3 3 3 3 4 4 3 3 3 4 3 3 4 4 3 4 4 4 3 1 1 4 4 3 3 1 4 0 4 3 3 1 3 3 3 3 1 4 3 3 3 1 0 4 3 2 2 4 4 3 1 1 1 1 3 1 3 3 3 1 3 2 0 4 2 3 4 1 5 3 3 4 4 3 4 1 0 1 1 3 5 1 4 3 1 3 3 1 4 4 3 5 4 3 3 1 1 3 3 5 3 1 3 4 0 2 3 4 4 1 1 4 2 4 1 3 3 3 4 0 3 3 2 3 3 1 1 4 5 3 5 3 3 5 1 3 3 4 4 1 4 3 3 3 3 1 4 3 4 3 1 3 3 1 2 0 4 1 0 1 1 4 5 0 4 2 2 1 1 4 4 4 2 3 4 3 0 4 5 4 3 3 4 0 4 1 3 1 3 1 0 3 1 3 4 0 3 4 1 2 5 2 5 3 1 5 3 1 5 3 1 5 3 3 3 0 3 3 3 4 1 1 3 4 2 3 1 4 5 3 5 3 2 5 3 3 1 4 1 2 3 5 5 3 3 3 5 1 3 1 3 0 5 1 5 4 1 3 3 3 5 1 1 3 3 2 4 1 3 4 3 5 2 4 5 2 1 0 4 1 4 3 3 3 1 2 1 1 1 1 2 0 4 4 2 1 0 1 3 5 4 3 4 4 3 1 3 1 4 4 3 3 3 4 3 4 4 1 3 1 1 3 3 3 3 4 4 3 0 3 4 5 2 4 3 3 3 1 4 3 2 4 1 3 3 2 5 4 4 1 1 3 4 3 4 3 4 1 3 3 2 4 4 3 4 1 5 4 1 1 3 1 3 2 1 3 1 3 1 1 4 3 4 3 1 1 5 1 3 4 4 4 1 3 1 3 3 4 1 0 3 3 3 1 3 3 2 3 3 3 3 1 2 0 3 1 3 3 3 3 3 3 3 4 3 4 

In [15]:
from sklearn import metrics 
print(metrics.confusion_matrix(Y,Pred))

[[ 48  14   7   1   2   4]
 [ 21 115  36  35  28  25]
 [  3  16  68   7   8   6]
 [ 12  66  61 141  69  49]
 [  6  33  32  69 129  55]
 [  2   6  10  14  13  72]]


In [16]:
target_names = ['Pants', 'False', 'Barely-True','Hlaf-True','Mostly-True','True']

print(metrics.classification_report(Y, Pred,target_names =target_names))

              precision    recall  f1-score   support

       Pants       0.52      0.63      0.57        76
       False       0.46      0.44      0.45       260
 Barely-True       0.32      0.63      0.42       108
   Hlaf-True       0.53      0.35      0.42       398
 Mostly-True       0.52      0.40      0.45       324
        True       0.34      0.62      0.44       117

    accuracy                           0.45      1283
   macro avg       0.45      0.51      0.46      1283
weighted avg       0.48      0.45      0.45      1283

